### Tranformaciones y Join ###



In [4]:
from pyspark.sql import SparkSession

# Cerrar sesión actual
try:
    spark.stop()
    print("✅ Sesión anterior cerrada")
except:
    print("ℹ️ No había sesión previa")

# Reiniciar con configuración explícita del JAR
spark = (SparkSession.builder
    .appName("Validacion-incremental")
    .master("spark://spark-master:7077")
    .config("spark.executor.instances", "1")
    .config("spark.executor.cores", "1")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "1g")
    .config("spark.jars", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .config("spark.driver.extraClassPath", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .config("spark.executor.extraClassPath", "/opt/spark/jars/mssql-jdbc-13.2.0.jre8.jar")
    .enableHiveSupport()
    .getOrCreate())

print("✅ Spark version:", spark.version)

ℹ️ No había sesión previa


25/09/15 15:22:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✅ Spark version: 2.4.5


In [ ]:
df_cliente= spark.read.parquet("hdfs://namenode:8020/bronze/clientes")
df_pedidos= spark.read.parquet("hdfs://namenode:8020/bronze/df_pedidos")
df_cliente.show(30)

+---------+--------------+--------------------+----------+--------------------+--------------------+
|ClienteID|        Nombre|               Email|  Telefono|          CreateTime|          UpdateTime|
+---------+--------------+--------------------+----------+--------------------+--------------------+
|        1|    Juan Pérez|juan.perez@email.com| 089000100|2025-08-24 21:27:...|2025-08-24 21:31:...|
|        2|   María López|maria22.nueva@ema...|0987654321|2025-08-24 21:27:...|2025-08-24 21:31:...|
|        3|   Carlos Ruiz|carlos.ruiz@email...|0971122334|2025-08-24 21:27:...|                null|
|        4|    Ana Torres|ana.torres@email.com|0965544332|2025-08-24 21:27:...|                null|
|        5|   Pedro Gómez|pedro.gomez@email...|0956677889|2025-08-24 21:27:...|                null|
|        7|     leo Pérez| leo.perez@email.com|0991234522|2025-08-29 19:50:...|                null|
|       12|          dary|  dary.pez@email.com|0991234567|2025-09-10 22:19:...|            

In [8]:
df_pedidos= spark.read.parquet("hdfs://namenode:8020/bronze/df_pedidos")
df_pedidos.show(30)

+--------+---------+------+-----------+--------------------+----------+
|PedidoID|ClienteID| Monto|FechaPedido|          CreateTime|UpdateTime|
+--------+---------+------+-----------+--------------------+----------+
|       1|        1|150.50| 2025-09-01|2025-09-14 19:36:...|      null|
|       2|        2|200.00| 2025-09-02|2025-09-14 19:36:...|      null|
|       3|        3| 50.00| 2025-09-03|2025-09-14 19:36:...|      null|
|       4|        4|300.00| 2025-09-04|2025-09-14 19:36:...|      null|
|       5|        5|120.00| 2025-09-05|2025-09-14 19:36:...|      null|
|       6|        1| 80.00| 2025-09-06|2025-09-14 19:36:...|      null|
|       7|        2| 60.00| 2025-09-07|2025-09-14 19:36:...|      null|
|       8|        3| 90.00| 2025-09-08|2025-09-14 19:36:...|      null|
|       9|        4|110.00| 2025-09-09|2025-09-14 19:36:...|      null|
|      10|        5| 75.00| 2025-09-10|2025-09-14 19:36:...|      null|
|      11|        7| 95.00| 2025-09-11|2025-09-14 19:36:...|    

In [18]:
from pyspark.sql.functions import broadcast, col
join_pedidos_df = df_pedidos.alias("p").join(broadcast(df_cliente.alias("c")), on="ClienteID", how="inner").\
select(
        col("p.PedidoID"),
        col("p.ClienteID"),
        col("c.Nombre"),
        col("c.Email"),
        col("p.Monto"),
        col("p.FechaPedido"),
        col("p.CreateTime").alias("PedidoCreateTime"),
        col("c.CreateTime").alias("ClienteCreateTime")
    )

In [19]:
join_pedidos_df.show()

+--------+---------+--------------+--------------------+------+-----------+--------------------+--------------------+
|PedidoID|ClienteID|        Nombre|               Email| Monto|FechaPedido|    PedidoCreateTime|   ClienteCreateTime|
+--------+---------+--------------+--------------------+------+-----------+--------------------+--------------------+
|       1|        1|    Juan Pérez|juan.perez@email.com|150.50| 2025-09-01|2025-09-14 19:36:...|2025-08-24 21:27:...|
|       2|        2|   María López|maria22.nueva@ema...|200.00| 2025-09-02|2025-09-14 19:36:...|2025-08-24 21:27:...|
|       3|        3|   Carlos Ruiz|carlos.ruiz@email...| 50.00| 2025-09-03|2025-09-14 19:36:...|2025-08-24 21:27:...|
|       4|        4|    Ana Torres|ana.torres@email.com|300.00| 2025-09-04|2025-09-14 19:36:...|2025-08-24 21:27:...|
|       5|        5|   Pedro Gómez|pedro.gomez@email...|120.00| 2025-09-05|2025-09-14 19:36:...|2025-08-24 21:27:...|
|       6|        1|    Juan Pérez|juan.perez@email.com|

In [20]:
join_pedidos_df.write.mode("overwrite").parquet("hdfs://namenode:8020/silver/pedidos")

In [22]:
df= spark.read.parquet("hdfs://namenode:8020/silver/pedidos")
df.show()

+--------+---------+--------------+--------------------+------+-----------+--------------------+--------------------+
|PedidoID|ClienteID|        Nombre|               Email| Monto|FechaPedido|    PedidoCreateTime|   ClienteCreateTime|
+--------+---------+--------------+--------------------+------+-----------+--------------------+--------------------+
|       1|        1|    Juan Pérez|juan.perez@email.com|150.50| 2025-09-01|2025-09-14 19:36:...|2025-08-24 21:27:...|
|       2|        2|   María López|maria22.nueva@ema...|200.00| 2025-09-02|2025-09-14 19:36:...|2025-08-24 21:27:...|
|       3|        3|   Carlos Ruiz|carlos.ruiz@email...| 50.00| 2025-09-03|2025-09-14 19:36:...|2025-08-24 21:27:...|
|       4|        4|    Ana Torres|ana.torres@email.com|300.00| 2025-09-04|2025-09-14 19:36:...|2025-08-24 21:27:...|
|       5|        5|   Pedro Gómez|pedro.gomez@email...|120.00| 2025-09-05|2025-09-14 19:36:...|2025-08-24 21:27:...|
|       6|        1|    Juan Pérez|juan.perez@email.com|

In [24]:
df.show(2)

+--------+---------+-----------+--------------------+------+-----------+--------------------+--------------------+
|PedidoID|ClienteID|     Nombre|               Email| Monto|FechaPedido|    PedidoCreateTime|   ClienteCreateTime|
+--------+---------+-----------+--------------------+------+-----------+--------------------+--------------------+
|       1|        1| Juan Pérez|juan.perez@email.com|150.50| 2025-09-01|2025-09-14 19:36:...|2025-08-24 21:27:...|
|       2|        2|María López|maria22.nueva@ema...|200.00| 2025-09-02|2025-09-14 19:36:...|2025-08-24 21:27:...|
+--------+---------+-----------+--------------------+------+-----------+--------------------+--------------------+
only showing top 2 rows



In [ ]:
#contar filas
df.count()

21

In [ ]:
#contar por grupos
df.groupBy("PedidoID").count().show()

+--------+-----+
|PedidoID|count|
+--------+-----+
|      12|    1|
|       1|    1|
|      13|    1|
|       6|    1|
|      16|    1|
|       3|    1|
|      20|    1|
|       5|    1|
|      19|    1|
|      15|    1|
|       9|    1|
|      17|    1|
|       4|    1|
|       8|    1|
|       7|    1|
|      10|    1|
|      11|    2|
|      14|    1|
|       2|    1|
|      18|    1|
+--------+-----+



In [32]:
len(df.columns)

8